In [1]:
import pandas as pd
from helper import create_lags, create_date_features, convert_to_category, create_forecast
import gc
import pickle

In [2]:
test = pd.read_parquet('data/test.snap.parquet')
test.tail()

c:\Users\lucas\anaconda3\envs\shipbob-interview\lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
853715,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.980469
853716,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.480469
853717,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,3.980469
853718,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,5,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.280273
853719,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,1,2016-06-19,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.000000


In [23]:
duplicates = test[test.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, item_id, dept_id, cat_id, store_id, state_id, d, y, date, wm_yr_wk, event_name_1, event_type_1, event_name_2, event_type_2, snap_CA, snap_TX, snap_WI, sell_price]
Index: []


In [3]:
test = test.reset_index(drop=True)

In [4]:
df_lag = test[['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'd', 'y', 'date']]

In [5]:
df_lagged = create_lags(df_lag, 28)

c:\Users\lucas\OneDrive\Documentos\codes\DS-CaseStudy-SalesForecast\helper.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'lag_{lag}'] = df.groupby(['id'])['y'].shift(lag)
c:\Users\lucas\OneDrive\Documentos\codes\DS-CaseStudy-SalesForecast\helper.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'lag_{lag}'] = df.groupby(['id'])['y'].shift(lag)
c:\Users\lucas\OneDrive\Documentos\codes\DS-CaseStudy-SalesForecast\helper.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or ob

In [6]:
df_lagged.drop(columns=['d','y'], inplace=True)
df_lagged.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,lag_1,lag_2,lag_3,...,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28
1612,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-05-23,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32102,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-05-24,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62592,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-05-25,0.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93082,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-05-26,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123572,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-05-27,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_date_features = create_date_features(test[['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'd', 'date']], 'date')
df_date_features.drop(columns=['d'], inplace=True)
df_date_features.head()

c:\Users\lucas\OneDrive\Documentos\codes\DS-CaseStudy-SalesForecast\helper.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
c:\Users\lucas\OneDrive\Documentos\codes\DS-CaseStudy-SalesForecast\helper.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'] = df[date_col].dt.day
c:\Users\lucas\OneDrive\Documentos\codes\DS-CaseStudy-SalesForecast\helper.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,id,item_id,dept_id,cat_id,store_id,state_id,date,day,month,year,dayofweek,is_weekend
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,23,5,2016,0,False
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,23,5,2016,0,False
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,23,5,2016,0,False
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,23,5,2016,0,False
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,23,5,2016,0,False


In [24]:
# Merge dataframes
merged_df = test.merge(df_lagged, on=['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'date'], how='inner')
merged_df = merged_df.merge(df_date_features, on=['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'date'], how='inner')

merged_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,...,lag_24,lag_25,lag_26,lag_27,lag_28,day,month,year,dayofweek,is_weekend
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,2,2016-05-23,11617,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,2016-05-23,11617,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,2016-05-23,11617,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,2016-05-23,11617,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,2,2016-05-23,11617,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False


In [25]:
duplicates = merged_df[merged_df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, item_id, dept_id, cat_id, store_id, state_id, d, y, date, wm_yr_wk, event_name_1, event_type_1, event_name_2, event_type_2, snap_CA, snap_TX, snap_WI, sell_price, lag_1, lag_2, lag_3, lag_4, lag_5, lag_6, lag_7, lag_8, lag_9, lag_10, lag_11, lag_12, lag_13, lag_14, lag_15, lag_16, lag_17, lag_18, lag_19, lag_20, lag_21, lag_22, lag_23, lag_24, lag_25, lag_26, lag_27, lag_28, day, month, year, dayofweek, is_weekend]
Index: []

[0 rows x 51 columns]


In [9]:
merged_df['d'].value_counts()

d
d_1942    30490
d_1943    30490
d_1944    30490
d_1945    30490
d_1946    30490
          ...  
d_966         0
d_967         0
d_968         0
d_969         0
d_100         0
Name: count, Length: 1969, dtype: int64

In [10]:
merged_df.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date'], inplace=True)

In [11]:
merged_df.tail()

,,,,,,,d,y,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,...,lag_24,lag_25,lag_26,lag_27,lag_28,day,month,year,dayofweek,is_weekend
id,item_id,dept_id,cat_id,store_id,state_id,date,,,,,,,,,,,,,,,,,,,,,
FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,2016-06-19,d_1969,0,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,...,1.0,1.0,0.0,0.0,NaN,19,6,2016,6,True
FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,2016-06-19,d_1969,0,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,...,1.0,1.0,0.0,0.0,NaN,19,6,2016,6,True
FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,2016-06-19,d_1969,0,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,...,0.0,0.0,0.0,1.0,NaN,19,6,2016,6,True
FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,2016-06-19,d_1969,5,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,...,0.0,0.0,0.0,0.0,NaN,19,6,2016,6,True
FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,2016-06-19,d_1969,1,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,...,1.0,1.0,5.0,0.0,NaN,19,6,2016,6,True


In [12]:
true_value = merged_df['y']
merged_df.drop(columns=['y'], inplace=True)

merged_df = convert_to_category(merged_df, ['d', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_weekend'])

In [13]:
merged_df.head()

c:\Users\lucas\anaconda3\envs\shipbob-interview\lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
c:\Users\lucas\anaconda3\envs\shipbob-interview\lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,,,,,,,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,...,lag_24,lag_25,lag_26,lag_27,lag_28,day,month,year,dayofweek,is_weekend
id,item_id,dept_id,cat_id,store_id,state_id,date,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,d_1942,11617,nan,nan,nan,nan,0,0,0,8.382812,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,d_1942,11617,nan,nan,nan,nan,0,0,0,3.970703,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,d_1942,11617,nan,nan,nan,nan,0,0,0,2.970703,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,d_1942,11617,nan,nan,nan,nan,0,0,0,4.640625,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False
HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-23,d_1942,11617,nan,nan,nan,nan,0,0,0,2.880859,...,NaN,NaN,NaN,NaN,NaN,23,5,2016,0,False


In [14]:
gc.collect()

30

In [15]:
prediction = create_forecast(df=merged_df, model_name="best_model.pkl", features=merged_df.columns)


In [16]:
prediction.head(30)

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,0,...,3,2,2,2,2,2,2,2,2,2
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,0,...,1,1,1,1,1,0,0,0,0,0
5,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,2,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
6,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,2,1,2,1,3,...,8,11,11,8,9,10,11,12,14,14
8,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
prediction.shape

(853720, 33)

In [18]:
prediction.columns

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1942',
       'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949',
       'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956',
       'd_1957', 'd_1958', 'd_1959', 'd_1960', 'd_1961', 'd_1962', 'd_1963',
       'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969'],
      dtype='object')

In [19]:
prediction.index

RangeIndex(start=0, stop=853720, step=1)

In [20]:
prediction.tail()

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
853715,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
853716,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
853717,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
853718,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,2,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
853719,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,2,0,2,1,1,...,1,1,1,1,1,1,1,1,2,1


In [21]:
duplicates = prediction[prediction.duplicated()]
print(duplicates)

              item_id    dept_id   cat_id store_id state_id  d_1942  d_1943  \
30490   HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1       CA       0       0   
30491   HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1       CA       0       0   
30492   HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1       CA       1       0   
30493   HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1       CA       1       0   
30494   HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1       CA       1       0   
...               ...        ...      ...      ...      ...     ...     ...   
853715    FOODS_3_823    FOODS_3    FOODS     WI_3       WI       1       0   
853716    FOODS_3_824    FOODS_3    FOODS     WI_3       WI       1       0   
853717    FOODS_3_825    FOODS_3    FOODS     WI_3       WI       0       0   
853718    FOODS_3_826    FOODS_3    FOODS     WI_3       WI       2       0   
853719    FOODS_3_827    FOODS_3    FOODS     WI_3       WI       2       0   

        d_1944  d_1945  d_1946  ...  d_1960  d_1961

In [26]:
prediction.drop_duplicates(inplace=True)

In [27]:
prediction.shape

(30490, 33)

In [28]:
prediction

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,0,...,3,2,2,2,2,2,2,2,2,2
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,0,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30486,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30487,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
30488,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,2,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [29]:
prediction.to_csv('submission.csv', index=False)